In [ ]:
%cd /content/drive/MyDrive/justnlp/summarization

/content/drive/MyDrive/justnlp/summarization


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train Data

In [ ]:
import json
import pandas as pd
train_path='./dataset/train/train_judg.jsonl'
judg=[]
with open(train_path,'r',encoding='utf-8') as f:
  for line in f:
    obj=json.loads(line)
    judg.append(obj)
judg=pd.DataFrame(judg)

In [ ]:
judg.head()

,ID,Judgment
0,id_10,Case :- WRIT - C No. - 11383 of 2023\nPetition...
1,id_1000,Versus\nAppearance:\nand\nDate : 22/09/2023\n1...
2,id_1001,Non-Reportable\nCriminal Appeal No._________ o...
3,id_1002,---- Appellant\nVersus\nUmesh Sharma S/o Late...
4,id_1003,1Whether Reporters of Local Papers may be allo...


In [ ]:
judg.shape

(1200, 2)

In [ ]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
judg["word_count"] = judg["Judgment"].apply(lambda x: len(nltk.word_tokenize(x)))
judg["sentence_count"] = judg["Judgment"].apply(lambda x: len(nltk.sent_tokenize(x)))
print(judg[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10        3949             258
1     id_1000        1938              51
2     id_1001        1574              56
3     id_1002        2386              79
4     id_1003        2008              60
...       ...         ...             ...
1195   id_995        8844             309
1196   id_996        9094             346
1197   id_997        1135              33
1198   id_998        2194              80
1199   id_999        2854             100

[1200 rows x 3 columns]


In [ ]:
judg['word_count'].min()

169

In [ ]:
judg['word_count'].max()

149745

In [ ]:
judg['word_count'].mean()

np.float64(8368.1275)

In [ ]:
judg['Judgment'].isnull().sum()

np.int64(0)

In [ ]:
import re
judg["clean_text"] = judg["Judgment"].str.lower()
#judg["clean_text"] = judg["clean_text"].apply(lambda x: re.sub(r"[^a-z0-9\s]", "", x))
judg["clean_text"] = judg["clean_text"].apply(lambda x: re.sub(r"\s+", " ", x))
print(judg[["ID", "clean_text"]])



           ID                                         clean_text
0       id_10  case :- writ - c no. - 11383 of 2023 petitione...
1     id_1000  versus appearance: and date : 22/09/2023 1.the...
2     id_1001  non-reportable criminal appeal no._________ of...
3     id_1002  ---- appellant versus umesh sharma s/o late o...
4     id_1003  1whether reporters of local papers may be allo...
...       ...                                                ...
1195   id_995  (arising out of special leave petition ( crimi...
1196   id_996  % reserved on: 15.03.2023 pronounced on: 16.05...
1197   id_997  reportable writ petition (c) no 961 of 2021 ne...
1198   id_998  m/s. biovet private limited ….applicants in th...
1199   id_999  nc: 2023:khc-d:13177-db wa no. 100406 of 2023 ...

[1200 rows x 2 columns]


In [ ]:
judg["clean_text"].apply(lambda x: len(nltk.word_tokenize(x))).min()

169